# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.io import boreholes_from_files, striplog_from_text,gdf_viewer,\
df_from_sources, striplog_from_df
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
search_dir=ROOT_DIR+'/CF_data/synthese/Result_traitem/'
cols=['ID','X','Y','Z','Long_Pz','Diam_Pz_int']
filename='source_Boreholes'

In [3]:
df_all=df_from_sources(search_dir, filename, cols, verbose=False)

--> base_Memoris3/source_Boreholes.csv: (148 lines)
--> profils_sols_donnees_forages/source_Boreholes.csv: (42 lines)

The overall dataframe contains 190 lines. 2 files used


In [4]:
striplog_from_df(df_all[:5])

{'1': Striplog(1 Intervals, start=0.0, stop=3.0),
 '12': Striplog(1 Intervals, start=0.0, stop=3.0),
 '22': Striplog(1 Intervals, start=0.0, stop=3.0),
 '25': Striplog(1 Intervals, start=0.0, stop=3.0),
 '32': Striplog(1 Intervals, start=0.0, stop=3.0)}

In [5]:
gdf_viewer(df_all)

Rows : 190, columns : 7


interactive(children=(IntSlider(value=10, description='rows', max=190, min=10, readout=False), IntSlider(value…

In [6]:
df_all.query('ID=="1"')

,ID,X,Y,Z,Long_Pz,Diam_Pz_int,geometry
0,1,153124.071,122653.382,102.622,6.941,0.025,POINT (153124.071 122653.382)


In [7]:
dist_crit=1

a=0
for i in range(0,len(df_all)-1):
    d=df_all.loc[i, 'geometry'].distance(df_all.loc[i+1, 'geometry'])
    if d<dist_crit or d>1e+4:
        a+=1
        #print(f"{df_all.loc[i,'ID']} <-> {df_all.loc[i+1,'ID']} : {d:.2} meters")
print('\n',f'{a}/{len(df_all)} elements with bad XY')


 29/190 elements with bad XY


In [8]:
import random
# the idea below can be used to build a french lexicon or select color/hatches from a list
test=df_all[:5].copy()
test['Litho']=''
l=['sand', 'clay', 'limestone', 'coal'] #funny random lithology :)
c=['white','red', 'blue', 'black'] 
test['Litho']=test['Litho'].apply(lambda x:random.sample(c,1)[0]+' '+random.sample(l,1)[0])

In [9]:
test=test.append({'ID':'1','X':153124.071,'Y':122653.382,'Z':101.983, 'Long':10, 'Litho':'blue limestone'}, ignore_index=True)
test

,ID,X,Y,Z,Long_Pz,Diam_Pz_int,geometry,Litho,Long
0,1,153124.071,122653.382,102.622,6.941,0.025,POINT (153124.071 122653.382),white limestone,NaN
1,12,153021.795,122640.338,102.899,3.271,0.041,POINT (153021.795 122640.338),blue coal,NaN
2,22,152941.137,122615.181,102.207,6.043,0.025,POINT (152941.137 122615.181),white coal,NaN
3,25,152997.314,122604.861,102.058,6.512,0.025,POINT (152997.314 122604.861),blue sand,NaN
4,32,153046.317,122603.273,101.870,6.730,0.025,POINT (153046.317 122603.273),black coal,NaN
5,1,153124.071,122653.382,101.983,NaN,NaN,None,blue limestone,10.0


## Create Boreholes from dataframes or files

In [10]:
df_all

,ID,X,Y,Z,Long_Pz,Diam_Pz_int,geometry
0,1,153124.071,122653.382,102.622,6.941,0.025,POINT (153124.071 122653.382)
1,12,153021.795,122640.338,102.899,3.271,0.041,POINT (153021.795 122640.338)
2,22,152941.137,122615.181,102.207,6.043,0.025,POINT (152941.137 122615.181)
3,25,152997.314,122604.861,102.058,6.512,0.025,POINT (152997.314 122604.861)
4,32,153046.317,122603.273,101.870,6.730,0.025,POINT (153046.317 122603.273)
...,...,...,...,...,...,...,...
185,P2Msup,NaN,NaN,NaN,NaN,NaN,POINT (nan nan)
186,P4Msup,NaN,NaN,NaN,NaN,NaN,POINT (nan nan)
187,P12Msup,NaN,NaN,NaN,NaN,NaN,POINT (nan nan)
188,P537,NaN,NaN,NaN,NaN,NaN,POINT (nan nan)


In [11]:
bhs_df=[df_all]

In [12]:
boreholes, components= boreholes_from_files(bhs_df, verbose=True, diam_field='Diam_Pz_int', length_field='Long_Pz')#, x=list(df_all.X), y=list(df_all.Y), verbose=False)


Dataframe 0 processing...
Length: 0      6.941
1      3.271
2      6.043
3      6.512
4      6.730
       ...  
185      NaN
186      NaN
187      NaN
188      NaN
189      NaN
Name: Long_Pz, Length: 190, dtype: float64
|__ID:'1' -- No length provided, treated with default (length=3.0)
|__ID:'1' -- No lithology data, treated with default ('white sand')
{0: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f974747d940>, 'base': <core.orm.PositionOrm object at 0x7f9747416370>}}

|__ID:'12' -- No length provided, treated with default (length=3.0)
|__ID:'12' -- No lithology data, treated with default ('white sand')
{1: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f974742b5e0>, 'base': <core.orm.PositionOrm object at 0x7f974742b190>}}

|__ID:'22' -- No length provided, treated with default (length=3.0)
|__ID:'22' -- No lithology data, treated with default ('white sand')
{2: {'description': 'white s

|__ID:'170' -- No length provided, treated with default (length=3.0)
|__ID:'170' -- No lithology data, treated with default ('white sand')
{33: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97473e9c40>, 'base': <core.orm.PositionOrm object at 0x7f97473e9550>}}

|__ID:'172' -- No length provided, treated with default (length=3.0)
|__ID:'172' -- No lithology data, treated with default ('white sand')
{34: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97473e0820>, 'base': <core.orm.PositionOrm object at 0x7f97473ee370>}}

|__ID:'175' -- No length provided, treated with default (length=3.0)
|__ID:'175' -- No lithology data, treated with default ('white sand')
{35: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97473eea00>, 'base': <core.orm.PositionOrm object at 0x7f97473ee9a0>}}

|__ID:'180' -- No length provided, treated with default (length=3.0

|__ID:'FP14sup' -- No length provided, treated with default (length=3.0)
|__ID:'FP14sup' -- No lithology data, treated with default ('white sand')
{72: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97473affa0>, 'base': <core.orm.PositionOrm object at 0x7f97473afd60>}}

|__ID:'FP15' -- No length provided, treated with default (length=3.0)
|__ID:'FP15' -- No lithology data, treated with default ('white sand')
{73: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97473b68e0>, 'base': <core.orm.PositionOrm object at 0x7f97473b6940>}}

|__ID:'FP160PROF' -- No length provided, treated with default (length=3.0)
|__ID:'FP160PROF' -- No lithology data, treated with default ('white sand')
{74: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97473b6310>, 'base': <core.orm.PositionOrm object at 0x7f97473b63d0>}}

|__ID:'FP160SUP' -- No length provided, treat

|__ID:'543' -- No length provided, treated with default (length=3.0)
|__ID:'543' -- No lithology data, treated with default ('white sand')
{106: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f9747360f70>, 'base': <core.orm.PositionOrm object at 0x7f9747360eb0>}}

|__ID:'544' -- No length provided, treated with default (length=3.0)
|__ID:'544' -- No lithology data, treated with default ('white sand')
{107: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f974736eee0>, 'base': <core.orm.PositionOrm object at 0x7f974736e3a0>}}

|__ID:'545' -- No length provided, treated with default (length=3.0)
|__ID:'545' -- No lithology data, treated with default ('white sand')
{108: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f9747371100>, 'base': <core.orm.PositionOrm object at 0x7f9747371d00>}}

|__ID:'541sup Al' -- No length provided, treated with default (l

|__ID:'DRAIN 9' -- No length provided, treated with default (length=3.0)
|__ID:'DRAIN 9' -- No lithology data, treated with default ('white sand')
{138: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f9747332df0>, 'base': <core.orm.PositionOrm object at 0x7f9747332b50>}}

|__ID:'DRAIN 10' -- No length provided, treated with default (length=3.0)
|__ID:'DRAIN 10' -- No lithology data, treated with default ('white sand')
{139: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f9747312520>, 'base': <core.orm.PositionOrm object at 0x7f97473395b0>}}

|__ID:'DRAIN 11' -- No length provided, treated with default (length=3.0)
|__ID:'DRAIN 11' -- No lithology data, treated with default ('white sand')
{140: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f9747339cd0>, 'base': <core.orm.PositionOrm object at 0x7f9747339b50>}}

|__ID:'DRAIN 12' -- No length provid

|__ID:'F17aM' -- No length provided, treated with default (length=3.0)
|__ID:'F17aM' -- No lithology data, treated with default ('white sand')
{169: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97472d8700>, 'base': <core.orm.PositionOrm object at 0x7f97472d8d60>}}

|__ID:'F17bM' -- No length provided, treated with default (length=3.0)
|__ID:'F17bM' -- No lithology data, treated with default ('white sand')
{170: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97472f2730>, 'base': <core.orm.PositionOrm object at 0x7f97472f20a0>}}

|__ID:'F17cM' -- No length provided, treated with default (length=3.0)
|__ID:'F17cM' -- No lithology data, treated with default ('white sand')
{171: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f97472e99d0>, 'base': <core.orm.PositionOrm object at 0x7f97472e9be0>}}

|__ID:'F17dM' -- No length provided, treated with de

In [13]:
boreholes

[<core.orm.BoreholeOrm>(id=1, length=6.941000000000004, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=12, length=3.271000000000001, diameter=0.041, intervals=1),
 <core.orm.BoreholeOrm>(id=22, length=6.043000000000006, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=25, length=6.511999999999992, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=32, length=6.729999999999995, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=34, length=6.711999999999998, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=44, length=6.636000000000009, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=58, length=4.343000000000004, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=59, length=6.943999999999994, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=64, length=6.797000000000009, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=65, length=6.625000000000004, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=67, length=6.8859

In [14]:
len(boreholes), len(components)

(190, 1)

## Display the striplog and 3D view

### Display project boreholes

In [15]:
remove('tmp_files/tfe_orm_db.db')

In [16]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [17]:
Base.metadata.create_all(engine)

2021-03-25 20:23:54,755 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-25 20:23:54,755 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:54,757 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-25 20:23:54,758 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:54,759 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2021-03-25 20:23:54,760 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:54,764 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2021-03-25 20:23:54,766 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:54,770 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2021-03-25 20:23:54,771 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:54,772 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2021-03-25 20:23:54,774 INFO sqlalchemy.engine.base.Engine ()
2021-03-25

In [18]:
Session = sessionmaker(bind=engine)
session = Session()

In [19]:
p = Project(session)
p.add_components(components)

2021-03-25 20:23:54,871 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:23:54,874 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:23:54,875 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:54,887 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-03-25 20:23:54,888 INFO sqlalchemy.engine.base.Engine (0, 'Sand, white')
2021-03-25 20:23:54,891 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:23:54,896 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:23:54,899 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:23:54,906 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the project

In [20]:
for bh in boreholes:
    p.add_borehole(bh)

2021-03-25 20:23:54,925 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-03-25 20:23:54,927 INFO sqlalchemy.engine.base.Engine ('1', 6.941000000000004, 0.025)
2021-03-25 20:23:54,932 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:23:54,934 INFO sqlalchemy.engine.base.Engine ((0, 0.0, 0.0, 0.0, 153124.071, 122653.382), (1, 3.0, 3.0, 3.0, 153124.071, 122653.382))
2021-03-25 20:23:54,940 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:23:54,942 INFO sqlalchemy.engine.base.Engine (0, '1', 0, 'white sand', 0, 1)
2021-03-25 20:23:54,945 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:23:54,953 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:23:54,955 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Borehol

2021-03-25 20:23:55,674 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-03-25 20:23:55,677 INFO sqlalchemy.engine.base.Engine (3.0, '22')
2021-03-25 20:23:55,679 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-03-25 20:23:55,681 INFO sqlalchemy.engine.base.Engine ('25', 6.511999999999992, 0.025)
2021-03-25 20:23:55,684 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:23:55,685 INFO sqlalchemy.engine.base.Engine (3, '25', 0, 'white sand', 6, 7)
2021-03-25 20:23:55,687 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:23:55,692 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:23:55,693 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM 

2021-03-25 20:23:56,350 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:23:56,352 INFO sqlalchemy.engine.base.Engine ((12, 0.0, 0.0, 0.0, 153149.254, 122591.79), (13, 3.0, 3.0, 3.0, 153149.254, 122591.79))
2021-03-25 20:23:56,355 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:23:56,357 INFO sqlalchemy.engine.base.Engine (6, '44', 0, 'white sand', 12, 13)
2021-03-25 20:23:56,359 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:23:56,371 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:23:56,373 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:23:56,375 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:56,380 INFO sqlalch

2021-03-25 20:23:57,353 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-03-25 20:23:57,354 INFO sqlalchemy.engine.base.Engine ('64', 6.797000000000009, 0.025)
2021-03-25 20:23:57,360 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:23:57,361 INFO sqlalchemy.engine.base.Engine (9, '64', 0, 'white sand', 18, 19)
2021-03-25 20:23:57,367 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:23:57,377 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:23:57,379 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:23:57,380 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:57,384 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals"

2021-03-25 20:23:58,100 INFO sqlalchemy.engine.base.Engine ('74', 6.931000000000003, 0.025)
2021-03-25 20:23:58,102 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:23:58,103 INFO sqlalchemy.engine.base.Engine (12, '74', 0, 'white sand', 24, 25)
2021-03-25 20:23:58,106 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:23:58,110 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:23:58,112 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:23:58,113 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:23:58,117 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS 

...

2021-03-25 20:24:37,701 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-03-25 20:24:37,704 INFO sqlalchemy.engine.base.Engine (3.0, 'P15bM')
2021-03-25 20:24:37,706 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-03-25 20:24:37,707 INFO sqlalchemy.engine.base.Engine ('P16M', 4.85, nan)
2021-03-25 20:24:37,710 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:24:37,711 INFO sqlalchemy.engine.base.Engine (175, 'P16M', 0, 'white sand', 350, 351)
2021-03-25 20:24:37,716 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:24:37,729 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:24:37,735 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "B

2021-03-25 20:24:38,545 INFO sqlalchemy.engine.base.Engine (3.0, 'P3M')
2021-03-25 20:24:38,551 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-03-25 20:24:38,552 INFO sqlalchemy.engine.base.Engine ('P2M', nan, nan)
2021-03-25 20:24:38,554 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:24:38,557 INFO sqlalchemy.engine.base.Engine (178, 'P2M', 0, 'white sand', 356, 357)
2021-03-25 20:24:38,566 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:24:38,573 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:24:38,575 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:24:38,576 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:24:38,583 INFO sqlalchemy.engine.ba

2021-03-25 20:24:39,339 INFO sqlalchemy.engine.base.Engine ((362, 0.0, 0.0, 0.0, nan, nan), (363, 3.0, 3.0, 3.0, nan, nan))
2021-03-25 20:24:39,341 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:24:39,342 INFO sqlalchemy.engine.base.Engine (181, 'P5M', 0, 'white sand', 362, 363)
2021-03-25 20:24:39,344 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:24:39,355 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:24:39,357 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:24:39,358 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:24:39,365 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_

2021-03-25 20:24:40,260 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-03-25 20:24:40,261 INFO sqlalchemy.engine.base.Engine (184, 'P11M', 0, 'white sand', 368, 369)
2021-03-25 20:24:40,267 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:24:40,275 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:24:40,277 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:24:40,279 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:24:40,289 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id 

2021-03-25 20:24:41,054 INFO sqlalchemy.engine.base.Engine (187, 'P12Msup', 0, 'white sand', 374, 375)
2021-03-25 20:24:41,057 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-25 20:24:41,066 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:24:41,067 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:24:41,069 INFO sqlalchemy.engine.base.Engine ()
2021-03-25 20:24:41,076 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:41,077 INFO sqlalchemy.engine.base.Engine ('P12Msup',)
2021-03-25 20:24:41,082

In [21]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

2021-03-25 20:24:41,771 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET length=? WHERE "Boreholes".id = ?
2021-03-25 20:24:41,776 INFO sqlalchemy.engine.base.Engine (3.0, 'P99')
2021-03-25 20:24:41,779 INFO sqlalchemy.engine.base.Engine COMMIT
Boreholes in the project :  190


In [22]:
p.boreholes

2021-03-25 20:24:41,808 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-25 20:24:41,810 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:41,811 INFO sqlalchemy.engine.base.Engine ('1',)
2021-03-25 20:24:41,824 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:41,826 INFO sqlalchemy.engine.base.Engine ('1',)
2021-03-25 20:24:41,830 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "B

2021-03-25 20:24:42,019 INFO sqlalchemy.engine.base.Engine ('65',)
2021-03-25 20:24:42,023 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:42,025 INFO sqlalchemy.engine.base.Engine ('67',)
2021-03-25 20:24:42,028 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:42,031 INFO sqlalchemy.engine.base.Engine ('67',)
2021-03-25 20:24:42,039 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Borehole

2021-03-25 20:24:42,193 INFO sqlalchemy.engine.base.Engine ('110',)
2021-03-25 20:24:42,197 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:42,199 INFO sqlalchemy.engine.base.Engine ('112',)
2021-03-25 20:24:42,206 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:42,207 INFO sqlalchemy.engine.base.Engine ('112',)
2021-03-25 20:24:42,210 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreh

2021-03-25 20:24:42,335 INFO sqlalchemy.engine.base.Engine ('168',)
2021-03-25 20:24:42,337 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:42,338 INFO sqlalchemy.engine.base.Engine ('170',)
2021-03-25 20:24:42,340 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:42,341 INFO sqlalchemy.engine.base.Engine ('170',)
2021-03-25 20:24:42,344 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreh

2021-03-25 20:24:42,476 INFO sqlalchemy.engine.base.Engine ('405',)
2021-03-25 20:24:42,478 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:42,480 INFO sqlalchemy.engine.base.Engine ('406',)
2021-03-25 20:24:42,484 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:42,485 INFO sqlalchemy.engine.base.Engine ('406',)
2021-03-25 20:24:42,488 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreh

...

2021-03-25 20:24:43,995 INFO sqlalchemy.engine.base.Engine ('F5M',)
2021-03-25 20:24:44,000 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:44,001 INFO sqlalchemy.engine.base.Engine ('F6M',)
2021-03-25 20:24:44,004 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:44,005 INFO sqlalchemy.engine.base.Engine ('F6M',)
2021-03-25 20:24:44,008 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreh

2021-03-25 20:24:44,139 INFO sqlalchemy.engine.base.Engine ('F14M',)
2021-03-25 20:24:44,141 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:44,142 INFO sqlalchemy.engine.base.Engine ('F15aM',)
2021-03-25 20:24:44,143 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:44,144 INFO sqlalchemy.engine.base.Engine ('F15aM',)
2021-03-25 20:24:44,150 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "

2021-03-25 20:24:44,271 INFO sqlalchemy.engine.base.Engine ('P16M',)
2021-03-25 20:24:44,276 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:44,277 INFO sqlalchemy.engine.base.Engine ('P17dM',)
2021-03-25 20:24:44,279 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:44,283 INFO sqlalchemy.engine.base.Engine ('P17dM',)
2021-03-25 20:24:44,290 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "

2021-03-25 20:24:44,407 INFO sqlalchemy.engine.base.Engine ('P4Msup',)
2021-03-25 20:24:44,411 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-03-25 20:24:44,412 INFO sqlalchemy.engine.base.Engine ('P12Msup',)
2021-03-25 20:24:44,419 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-03-25 20:24:44,423 INFO sqlalchemy.engine.base.Engine ('P12Msup',)
2021-03-25 20:24:44,426 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diamete

[<core.orm.BoreholeOrm>(id=1, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=12, length=3.0, diameter=0.041, intervals=1),
 <core.orm.BoreholeOrm>(id=22, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=25, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=32, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=34, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=44, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=58, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=59, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=64, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=65, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=67, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=74, length=3.0, diameter=0.025, intervals=1),
 <core.orm.BoreholeOrm>(id=80, length=3.0, diameter=

In [23]:
p.refresh(verbose=True)

2021-03-25 20:24:44,461 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-03-25 20:24:44,463 INFO sqlalchemy.engine.base.Engine ()


In [24]:
session.close()

2021-03-25 20:24:44,477 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [25]:
p.boreholes_3d[0]

Striplog(1 Intervals, start=0.0, stop=3.0)

### The Borehole in 3D using X3D for interaction within the notebook

In [26]:
p.plot3d(x3d=True)

##  ==================Zone Brouillon==================

#### Codes qui peuvent aider si besoin (Brouillon)